# User Defined Functions (UDFs)

## How does it work?!

![](img/xlwings_udf_architecture.png)

<div class="alert alert-info">
 
**Note**: xlwings works with all bitness combinations of Excel and Python, e.g. Python 64-bit and Excel 32-bit work happily together. 

</div>

# One-time Excel preparations for UDFs

1) To be able to import funtions: Enable `Trust access to the VBA project object model` under  

`File > Options > Trust Center > Trust Center Settings > Macro Settings`

2) Install the add-in if you haven't done so yet, see previous tutorial

# Our first UDF: Hello World

We stil have our `hello` project from the previous course module (if not: do `xlwings quickstart hello` again on the command prompt) with the following Python code:

In [ ]:
import xlwings as xw


def hello_xlwings():
    wb = xw.Book.caller()
    wb.sheets[0].range("A1").value = "Hello xlwings!"


@xw.func
def hello(name):
    return "hello {0}".format(name)

Now:
* click on the `Import Functions` button on the add-in (make sure a reference to xlwings is set, see previous tutorial) and
* call the function from a cell: `=hello("world")` or `=hello(A1)`

<div class="alert alert-info">
 
**Note**: After making changes to your function, (a) save the python file and (b) hit `Ctrl-Alt-F9` in Excel to recalculate the function and pick up the changes. You only need to re-import a formula when you change the arguments or the function name itself. 

</div>

# Array Formulas

In [ ]:
@xw.func
def add_one(x):
    return [[cell + 1 for cell in row] for row in x]

Now use it like this by selecting `D1:E2` and pressing `Ctrl+Shift+Enter` to create an array formula:

![](img/array_formula.png)

Remember that only 2d ranges come in as list of list? So this formula will fail on single cells and range vectors. Change it like this to work correctly:

In [ ]:
@xw.func
@xw.arg('x', ndim=2)
def add_one(x):
    return [[cell + 1 for cell in row] for row in x]

<div class="alert alert-info">

**Efficiency**: Use array formulas wherever possible instead of many single-cell formulas to speed up calculations. This is the same principle as we saw with the Range object: An array formula only crosses borders once.

</div>

# Converters and options: @xw.arg and @xw.ret decorators

`@xw.arg` (for arguments) and `@xw.ret` (for return values) are the equivalent of `mysheet.range.options()`:

## NumPy Converter

In [ ]:
import numpy as np

@xw.func
@xw.arg('x', np.array, ndim=2)
def add_one2(x):
    return x + 1

## Pandas DataFrame Converter

Pandas DataFrames are per default `ndim=2`, so no need to specify it explicitely.

In [ ]:
import pandas as pd

@xw.func
@xw.arg('x', pd.DataFrame, index=False, header=False)
@xw.ret(pd.DataFrame, index=False, header=False)
def add_one3(x):
    return x + 1

In this case, we could have also just used the `.values` method instead of `@xw.ret` to suppress the index and headers:

In [ ]:
@xw.func
@xw.arg('x', pd.DataFrame, index=False, header=False)
def add_one3(x):
    return (x + 1).values

## xw.Range Converter

Technically speaking, `xw.Range` is a "no-converter": It corresponds to what `mysheet.range(...)` returns:

In [ ]:
@xw.func
@xw.arg('x', xw.Range)
def get_formula(x):
    return x.formula

# Dynamic arrays

<div class="alert alert-info">

**Note**: The example here requires that you install the `quandl` package either with `conda install quandl` or `pip install quandl`

</div>

While array formulas are great in terms of efficiency, they are a bit cumbersome to work with as they need to be deleted/recreated whenever the dimensions change.
**Dynamic arrays** in xlwings get around that issue by providing a means of "writing outside the cell formula". 
  
**Note:** this is slightly against the design of Excel and thus you need to take care to not overwrite existing cell values by accident.
  
Lets fetch some data from Quandl, a data provider that offers free financial data:

`xlwings quickstart marketdata`

Put the following code into `marketdata.py`:

In [ ]:
import xlwings as xw
import quandl

# It's free, but only a few calls are allowed without API key
# quandl.ApiConfig.api_key = 'MY_QUANDL_API_KEY'

@xw.func
@xw.ret(expand='table')  # this makes it a dynamic array
def get_history(ticker, start_date=None, end_date=None):
    return quandl.get(ticker, start_date=start_date, end_date=end_date)

Now you can import the formula and use it with something like `=get_history("WIKI/AAPL", "2016-01-01")` to get the historical data for Apple.

<div class="alert alert-info">

**Note**: You must not use any volatile formula with dynamic arrays, like `=TODAY()`

</div>

# VBA Settings (add-in)

* `UDF Modules`: It allows you to specify various modules from where you want to import your functions. So you could do something like: 

  `common_functions;myproject`

  If you leave it empty, it expects a source file the way we've used it: in the same directory as the Excel file and with the same name but with a `.py` ending.

* Make sure python can find the directory where the source modules are, if necessary you need to tweak the PYTHONPATH.

* If you change code in python modules that are not directly imported from xlwings, using `Restart UDF Server` will load everything from scratch.

# In-Excel SQL Extension

Extensions are formulas directly embedded in the add-in and therefore available right after the installation of the add-in without further steps needed.

We have built in the sql extension: `=sql(SQL Statement, table a, table b, ...)`. Open the file `sql.xlsx` (no xlsm necessary!) and play around!

<div class="alert alert-info">

**Note**: Extensions don't require a macro-enabled workbook and don't require a reference to xlwings in the VBA editor neither!

</div>



# A Practical Example



Implement an `end_of_month` UDF that resamples daily time series into end-of-month time series. Use the `timeseries.xlsm` to get some time series data. See http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.resample.html

In [ ]:
import xlwings as xw
import pandas as pd

@xw.func
@xw.arg('x', pd.DataFrame)
def end_of_month(x):
    return x.resample('M').last()

# Asynchronous Functions

xlwings (since v0.14.0) offers an easy way to write asynchronous functions in Excel. Asynchronous functions return immediately with `#N/A waiting...` While the function is waiting for its return value, you can use Excel to do other stuff and whenever the return value is available, the cell value will be updated. This is the syntax:

In [ ]:
import xlwings as xw
import time

@xw.func(async_mode='threading')
def myfunction(a):
    time.sleep(5)  # long running tasks
    return a